In [1]:
AUTH_KEY = 'MDAzOTM4MDktY2Q0Ny00MjdjLThmYzgtMjlhNzk2N2IwYzcxOjJhYzE1YTYzLTQ5NjktNGM5OS05OTViLTc2OTkxNTZjNTQ2MA=='
CLIENT_ID = '00393809-cd47-427c-8fc8-29a7967b0c71'
MODEL_NAME = 'GigaChat'
SCOPE = 'GIGACHAT_API_PERS'

In [2]:
from langchain_community.chat_models import GigaChat
import os

params = {'credentials':AUTH_KEY, 'scope':SCOPE, 'verify_ssl_certs':False}

In [3]:
from model import embedder
from search import Searcher

C:\Users\sedir\AppData\Roaming\Python\Python311\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
C:\Users\sedir\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of RobertaModel were not initialized from the model checkpoint at ai-forever/ru-en-RoSBERTa and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
searcher = Searcher(model=embedder)
await searcher.load_questions()
await searcher.chunk_text_with_embeddings()

Batches: 100%|███████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  9.35it/s]


In [89]:
class GigaModel:
    def __init__(self, auth_key, scope, model_name, searcher):
        self.params = {'credentials':auth_key, 'scope':scope, 'verify_ssl_certs':False, 'model_name': model_name}
        self.llm = GigaChat(**self.params, temperature=0.01)
        self.searcher = searcher

    async def chat(self, text):
        try:
            res = self.llm.invoke(text)
            return res.content
        except Exception as e:
            print(e)
            return 'Error'

    async def get_rag_result(self, question):
        try:
            new_question = await giga.llm.ainvoke(f'A question has been sent to you from a user. If it\'s not accurate enough, rephrase it. Give me only new or old question.\nQuestion: {question}')
            new_question = new_question.content
            if len(new_question.split(':')) == 2:
                new_question = new_question.split(':')[-1].strip()
        except Exception as e:
            print(e)
            new_question = question

        chunks = [c[0] for c in await self.searcher.search_query(new_question, threshold_embed=0.5)]
        system_prompt = 'You are an assistant at the Preparatory Department for Foreign Students of the Ural Federal University. Your goal is to answer the questions that the applicants ask you.'
        documents = '\n'.join([f'Question: {c.metadata["question"]}\tAnswer: {c.page_content}' for c in chunks])
        task = f'The FAQ is presented to you, using this information, answer the question you asked.\nFAQ: {documents}'
        task = task[:15000] + f'\nQuestion: {new_question}'
        query = system_prompt + '\n' + task
        
        try:
            res = await self.llm.ainvoke(query)
            # print(query)
            return res.content
        except Exception as e:
            print(e)
            return ''

In [90]:
giga = GigaModel()

In [91]:
question = 'How much will the training cost?'

In [92]:
await giga.get_rag_result(question)

2024-11-27 21:30:27,372 - [INFO] - httpx - (_client.py)._send_single_request(1773) - HTTP Request: POST https://ngw.devices.sberbank.ru:9443/api/v2/oauth "HTTP/1.1 200 OK"
2024-11-27 21:30:27,374 - [INFO] - gigachat.client - (client.py)._aupdate_token(336) - OAUTH UPDATE TOKEN
2024-11-27 21:30:28,188 - [INFO] - httpx - (_client.py)._send_single_request(1773) - HTTP Request: POST https://gigachat.devices.sberbank.ru/api/v1/chat/completions "HTTP/1.1 200 OK"
Batches: 100%|███████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  8.33it/s]
2024-11-27 21:30:29,288 - [INFO] - httpx - (_client.py)._send_single_request(1773) - HTTP Request: POST https://gigachat.devices.sberbank.ru/api/v1/chat/completions "HTTP/1.1 200 OK"


You are an assistant at the Preparatory Department for Foreign Students of the Ural Federal University. Your goal is to answer the questions that the applicants ask you.
The FAQ is presented to you, using this information, answer the question you asked.
FAQ: Question: What are the tuition fees for programs at Ural Federal University?	Answer: Tuition fees at Ural Federal University depend on the program you choose. Please check the list of available programs in the [Programs and Courses](https:/urfu.ru/en/international/programs-and-courses/) section. All information regarding tuition fees can be found in this section.
Question: Are there any additional fees?	Answer: Please keep in mind that Ural Federal University does not claim any deposits for tuition fees or charge any additional registration fees for internal admission procedures (including application*, admission interviews, expert evaluation of educational documents etc).
If you are applying to a university through recruitment age

"The cost of the training program depends on the specific program you choose. To find out more about the tuition fees for different programs, please visit the [Programs and Courses](https://urfu.ru/en/international/programs-and-courses/) section on the university's website."